In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

titanic_df = pd.read_csv('train.csv')
'''
Passengerid : 탑승자 데이터 일련번호    int
survived : 0사망 1생존                 int
pclass : 1일등석 2이등석 3삼등석        int
sex : 성별                             object
name : 이름                            object
age : 나이                             float
sibsp : 동승한 형제자매 또는 배우자 수   int
parch : 동승한 부모님 또는 어린이 수     int
ticket : 티켓 번호                      object
fare : 요금                            float
cabin : 선실 번호                       object 
embarked : 중간 정착 항구 C, Q, S       object
'''

#Null 값 제거
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace = True)
titanic_df['Cabin'].fillna('N', inplace = True)
titanic_df['Embarked'].fillna('N', inplace = True)
titanic_df['Fare'].fillna(0, inplace=True)

#속성 변경 C32, C33, C34 => C
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]

#성별에 따른 생존률 확인
#sns.barplot(x = 'Sex', y = 'Survived', data=titanic_df)

#성별과 탑승석 종류에 따른 생존률 확인
#sns.barplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = titanic_df)

#연령대를 대분류로 구분
def get_age(age) :
    age_cat = ''
    if age <= -1 : age_cat = 'Unknown'
    elif age <= 5 : age_cat = 'Baby'
    elif age <= 12 : age_cat = 'Child'
    elif age <= 18 : age_cat = 'Teenager'
    elif age <= 25 : age_cat = 'Student'
    elif age <= 35 : age_cat = 'Young Adult'
    elif age <= 60 : age_cat = 'Adult'
    else : age_cat = 'Elderly'
    return age_cat

age_cat = ['Unknown', 'Baby', 'Chile', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : get_age(x))
#근데 굳이 할 필요는 없음.
titanic_df.drop('Age_cat', axis = 1, inplace=True)

#문자형 데이터를 숫자형으로 변환 (레이블 인코딩)
from sklearn import preprocessing

def encode_features(dataDF) :
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features :
        le = preprocessing.LabelEncoder()
        le = le.fit(dataDF[feature])
        dataDF[feature] = le.transform(dataDF[feature])
        
    return dataDF

titanic_df = encode_features(titanic_df)


#머신러닝에 불필요한 속성 제거
titanic_df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)

#X, y 구분 및 훈련, 테스트 셋 구분
y_titanic_df = titanic_df['Survived']
titanic_df.drop('Survived', axis = 1)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(titanic_df, y_titanic_df, test_size = 0.2, random_state = 1)
#참고로 랜덤 상태는 시드값으로 랜덤으로 만들어지는 훈련, 테스트 셋이 언제 만들어도 동일하게 하도록 하기 위함

#학습
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#결정트리
dt_clf = DecisionTreeClassifier(random_state = 1)
dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
print('결정 트리 \t정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

#랜덤포레스트
rf_clf = RandomForestClassifier(random_state = 1)
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
print('랜덤 포레스트\t정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

#로지스틱회귀
lr_clf = LogisticRegression(random_state = 1)
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
print('로지스틱 회귀 \t정확도 : {0:.4f}'.format(accuracy_score(y_test, lr_pred)))


#kFold 교차검증
from sklearn.model_selection import KFold

#함수로 하면 이렇게
'''
def exec_kfold(clf, folds = 5) :
    kfold = KFold(n_splits = folds)
    score = []
    
    for iter_count, (train_index, test_index) in enumerate(kfold.split(titanic_df)) :
        x_train, x_test = titanic_df.values[train_index], titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        clf.fit(x_train, y_train)
        predictions = clf.predict(x_test)
        accuracy = accuracy_score(y_test, predictions)
        score.append(accuracy)
        print("교차검증 {0} / 정확도{1:.4f}".format(iter_count, accuracy))
        
    mean_score = np.mean(score) #각 fold의 평균 계산
    print('평균 정확도 : {0:.4f}'.format(mean_score))

exec_kfold(dt_clf, folds = 5)
'''
#근데 굳이 함수로 할 필요 없음
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, titanic_df, y_titanic_df, cv = 5)
for iter_count, accuracy in enumerate(scores) :
    print("교차 검증 {} / 정확도 {:.4f}".format(iter_count, accuracy))
    
    
print('평균 정확도 : {:.4f}'.format(np.mean(scores)))

#GridSearchCV로 최적의 하이퍼 파라미터를 찾아 예측 성능 측정
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth' : [2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf' : [1,5,8]}
grid_dclf = GridSearchCV(dt_clf, param_grid = parameters, scoring='accuracy', cv=5)
grid_dclf.fit(x_train, y_train)

print('GridSearchCV 최적의 하이퍼 파라미터 : ', grid_dclf.best_params_)
print('GridSearchCV 최고 정확도 : {:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

#GridSearchCV의 최적 하이퍼 파라미터로 학습된 예측기로 예측 및 평가 수행
dpredictions = best_dclf.predict(x_test)
accuracy = accuracy_score(y_test, dpredictions)
print('테스트 세트에서의 결정트리 정확도 : {:.4f}'.format(accuracy))

결정 트리 	정확도 : 1.0000
랜덤 포레스트	정확도 : 1.0000
로지스틱 회귀 	정확도 : 1.0000
교차 검증 0 / 정확도 1.0000
교차 검증 1 / 정확도 1.0000
교차 검증 2 / 정확도 1.0000
교차 검증 3 / 정확도 1.0000
교차 검증 4 / 정확도 1.0000
평균 정확도 : 1.0000


C:\Users\mkjh9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV 최적의 하이퍼 파라미터 :  {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 1.0000
테스트 세트에서의 결정트리 정확도 : 1.0000
